# Clustering Zooniverse Marks to count Iguanas
Count all the iguanas in the images by clustering the marks from the zooniverse volunteers.
This does not compare the results to the gold standard and requires only a single file, the flattened zooniverse data export.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("./")
sys.path.append("./zooniverse")

## Intro
### Retrieve a Classification report from Zooniverse
This notebook is used to cluster the marks from the zooniverse volunteers to count the iguanas in the images saved in results/<phase_tag>/flat_dataset_filtered_Iguanas 3rd launch.csv

Used Methods are:

### DBSCAN 
It does not require the number of clusters to be specified. It is used here because, but has min_samples and eps as hyperparameters which need to be found. [Link](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
For finding eps and min_sample a simple **grid search** is used.
Additionally, DBSCAN not assume a specific shape for the clusters (K-means assumes clusters are gaussian in shape) even though we should assume that points around an iguana is gaussian shaped.

### HDBSCAN
It is an extension of DBSCAN which is more robust to hyperparameter settings as it finds epsilon and min_samples automatically. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html)

## Load the data

In [7]:
from pathlib import Path

import pandas as pd
from zooniverse.analysis import get_annotation_count_stats
from zooniverse.utils.filters import filter_df_user_threshold



# Phase Selection
phase_tag = "Iguanas 1st launch"
# phase_tag = "Iguanas 2nd launch"
# phase_tag = "Iguanas 3rd launch"


debug = False # debugging with a smaller dataset
plot_diagrams = False # plot the diagrams to disk for the clustering methods
show_plots = False # show the plots in the notebook

user_threshold = None # None or a number, filter records which have less than these user interactions.

use_gold_standard_subset = None # Use no filtering



## Input Path of all the data
input_path =Path("/Users/christian/data/zooniverse")
# input_path = Path("results/")
# Location for the analysis Results
output_path = Path(input_path.joinpath(f"2024_04_10_analysis").joinpath(phase_tag))
output_path.mkdir(exist_ok=True, parents=True)
reprocess = False # if True, the raw classification data is reprocessed. If False, the data is loaded from disk

# Location for plots
output_plot_path = output_path.joinpath("plots")
output_plot_path.mkdir(parents=True, exist_ok=True)


## Look into the config
This Config points to all files necessary for the analysis + the result files

In [8]:
from zooniverse.config import get_config_all

config = get_config_all(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config

{'annotations_source': PosixPath('/Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': None,
 'gold_standard_image_subset': None,
 'image_source': None,
 'yes_no_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_10_analysis/Iguanas 1st launch/yes_no_dataset_Iguanas 1st launch.csv'),
 'flat_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_10_analysis/Iguanas 1st launch/flat_dataset_Iguanas 1st launch.csv'),
 'flat_panoptes_points': PosixPath('/Users/christian/data/zooniverse/2024_04_10_analysis/Iguanas 1st launch/flat_panoptes_points_Iguanas 1st launch.csv'),
 'merged_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_10_analysis/Iguanas 1st launch/flat_dataset_filtered_Iguanas 1st launch.csv'),
 'comparison_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_10_analysis/Iguanas 1st launch/Iguanas 1st launch_method_comparison.csv'),
 'method_sums': PosixPath('/Users/chri

In [12]:
from zooniverse.utils.anonymize import UserAnonymizer
from zooniverse.utils.data_format import data_prep_all

if reprocess:
    ds_stats = data_prep_all(phase_tag=phase_tag, 
                         output_path=output_path, 
                         input_path=input_path, 
                         config=config)
    print(ds_stats)
    
    # TODO the data should be anonymized already
    # Anomymise the data to prevent usernames and user_ids to become public
    anonymizer = UserAnonymizer(config["flat_dataset"])
    anonymizer.anonymize_data()
    anonymizer.save_anonymized_data(config["flat_dataset"])
    
    anonymizer = UserAnonymizer(config["merged_dataset"])
    anonymizer.anonymize_data()
    anonymizer.save_anonymized_data(config["merged_dataset"])

In [13]:
if plot_diagrams == False:
    output_plot_path = None

# the flattened, filtered marks from zooniverse.
df_merged_dataset = pd.read_csv(config["merged_dataset"])

df_merged_dataset

,Unnamed: 0,flight_site_code,workflow_id,workflow_version,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name,mission_name,image_path,width,height
0,0,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,376.160156,84.558594,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
1,1,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,272.695312,297.894531,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
2,2,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,111.898438,410.011719,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
3,3,SFM1,14370,134.236,SFM01-1-2_264.jpg,47968406,462.800781,390.786377,Adult Male in a lek,Iguanas 1st launch,1878692.0,drewavery,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
4,4,SFM1,14370,134.236,SFM01-1-2_264.jpg,47968406,545.598145,423.652527,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,1878692.0,drewavery,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,3042,SFM1,14370,134.236,SFM01-2-2-1_120.jpg,47969236,406.704041,383.049408,"Could be an iguana, not sure",Iguanas 1st launch,NaN,not-logged-in-341c84c0e52bdc34f8f8,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,666,648
3043,3043,SFM1,14370,134.236,SFM01-2-2-1_120.jpg,47969236,577.184998,270.231110,"Could be an iguana, not sure",Iguanas 1st launch,NaN,not-logged-in-341c84c0e52bdc34f8f8,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,666,648
3044,3044,SFM1,14370,134.236,SFM01-1-2_210.jpg,47968359,316.314850,392.354614,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,2136122.0,H.axson,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
3045,3045,SFM1,14370,134.236,SFM01-1-2_210.jpg,47968359,326.542725,402.156372,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,2136122.0,H.axson,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630


In [14]:
df_flat_panoptes_points = pd.read_csv(config["flat_panoptes_points"])
df_flat_panoptes_points

,Unnamed: 0,classification_id,user_name,user_id,workflow_id,task,created_at,subject_id,image_name,x,y
0,161321,397130915,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-15 19:21:11 UTC,72332826,EGI01-1-2_83.jpg,297,102
1,161330,397130915,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-15 19:21:11 UTC,72332826,EGI01-1-2_83.jpg,297,102
2,103907,401114209,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-03-08 00:53:47 UTC,72332844,EGI01-2-1_26.jpg,1384,1027
3,103917,401114209,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-03-08 00:53:47 UTC,72332844,EGI01-2-1_26.jpg,1384,1027
4,199047,397739195,d84d31ab467c59e8a3ba49030312072e,003d2347111646e2fe7ae94b67bde94b,20600,T2,2022-02-18 20:16:43 UTC,72332945,EGI02-1_23.jpg,679,338
...,...,...,...,...,...,...,...,...,...,...,...
177227,108350,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,97,117
177228,108350,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,113,186
177229,108350,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,87,167
177230,108350,397662853,9432ca625cbcee771e54db4c895908d3,NaN,20600,T2,2022-02-18 12:38:57 UTC,72373351,ESCG02-2_65.jpg,443,124


## Look at the data


In [15]:
## Look at the data
df_merged_dataset


,Unnamed: 0,flight_site_code,workflow_id,workflow_version,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name,mission_name,image_path,width,height
0,0,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,376.160156,84.558594,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
1,1,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,272.695312,297.894531,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
2,2,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,111.898438,410.011719,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
3,3,SFM1,14370,134.236,SFM01-1-2_264.jpg,47968406,462.800781,390.786377,Adult Male in a lek,Iguanas 1st launch,1878692.0,drewavery,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
4,4,SFM1,14370,134.236,SFM01-1-2_264.jpg,47968406,545.598145,423.652527,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,1878692.0,drewavery,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,3042,SFM1,14370,134.236,SFM01-2-2-1_120.jpg,47969236,406.704041,383.049408,"Could be an iguana, not sure",Iguanas 1st launch,NaN,not-logged-in-341c84c0e52bdc34f8f8,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,666,648
3043,3043,SFM1,14370,134.236,SFM01-2-2-1_120.jpg,47969236,577.184998,270.231110,"Could be an iguana, not sure",Iguanas 1st launch,NaN,not-logged-in-341c84c0e52bdc34f8f8,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,666,648
3044,3044,SFM1,14370,134.236,SFM01-1-2_210.jpg,47968359,316.314850,392.354614,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,2136122.0,H.axson,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
3045,3045,SFM1,14370,134.236,SFM01-1-2_210.jpg,47968359,326.542725,402.156372,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,2136122.0,H.axson,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630


### Filter User if necessary and Marks


In [ ]:
print(f"Before filtering: {df_merged_dataset.subject_id.nunique()}")
# There images in which some people said there are iguanas, but then didn't mark them. Clustering with fewer than 3 dots doesn't make sense
if user_threshold is not None:
    print(f"filtering records which have less than {user_threshold} interactions.")
    df_merged_dataset = filter_df_user_threshold(df_merged_dataset, user_threshold=user_threshold)
    

from zooniverse.utils.filters import filter_remove_marks
# Check if partials are still in the data. There shouldn't be any
df_merged_dataset = filter_remove_marks(df_merged_dataset)


### Are there anonymous users in the data?
There should be some.

In [ ]:
df_merged_dataset[df_merged_dataset.user_id.isnull().values]

In [ ]:
# Amount of images
df_merged_dataset["subject_id"].nunique()

In [ ]:
## After filtering there
df_merged_dataset

In [ ]:
# how many marks per user
df_merged_dataset[["user_id", "x"]].groupby("user_id").count().head()

In [ ]:
# How many images are left in the zooniverse dataset?
df_merged_dataset["subject_id"].nunique()

## Clustering

### Basic Statics like mean, median, mode

In [ ]:
from zooniverse.analysis import get_mark_overview

basic_stats = []

for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
    annotations_count = get_mark_overview(df_image_name)

    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"])

    ### basic statistics like mean, median
    basic_stats.append(annotations_count_stats)
    

df_basic_stats = pd.DataFrame(basic_stats)    
df_comparison = df_basic_stats


In [ ]:
df_basic_stats

In [ ]:
# There might be records with too few annotations if they were not removed before
df_comparison[(df_comparison.sum_annotations_count < 5)].sort_values(by="users", ascending=False)

In [ ]:
# Fill NaN values with 0 because the errors can't be calculated otherwise
df_comparison.fillna(0, inplace=True)


In [ ]:
df_comparison

In [ ]:
df_comparison.sum()

### DBSCAN clustering and take the variant with the best silouette score for each image


In [ ]:
### The old variant
# from zooniverse.analysis import compare_dbscan
# 
# eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
# min_samples_variants = [3, 5, 8, 10]
# if debug:
#     eps_variants = [0.3]
#     min_samples_variants = [3]
# params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]
# 
# db_scan_results = {}
# db_scan_best_results = []
# db_scan_best_bic_results = []
# for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
# 
#     dbscan_localization = compare_dbscan(
#         # phase_tag=phase_tag,
#         params=params,
#         df_flat=df_image_name,
#         # output_path=output_path,
#         output_plot_path=output_plot_path,
#         plot=show_plots,
#         
#     )
# 
#     db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
#     db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_silouette_score", ascending=False).iloc[0])
#     
# df_dbscan_localization = pd.concat([*db_scan_results.values()])
# df_scan_best_results = pd.DataFrame(db_scan_best_results)



In [ ]:
# df_scan_best_results

In [ ]:
## fixes the problem with the silouette score sorting
from zooniverse.analysis import compare_dbscan

eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
min_samples_variants = [3, 5, 8, 10]
if debug:
    eps_variants = [0.3]
    min_samples_variants = [3]
params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]

db_scan_results = {}
db_scan_best_results = []
db_scan_best_bic_results = []
for image_name, df_image_name in df_merged_dataset.groupby("image_name"):

    dbscan_localization = compare_dbscan(
        params=params,
        df_flat=df_image_name,
        output_plot_path=output_plot_path,
        plot=show_plots
    )

    db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
    
    # DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
    # Sillouette Scoring needs a minimum of 2 clusters
    # if there are points in decent radius they will belong to a cluster
    if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
        db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
        db_scan_best_bic_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
        # If two or more cluster seem to exists take ones with the best Silouette score
    else:  
        # take the best result by silouette score if there are more clusters then 1
        db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values(["dbscan_silouette_score", "dbscan_count"], ascending=[False, False]).iloc[0])
    
df_dbscan_localization = pd.concat([*db_scan_results.values()])
df_scan_best_results = pd.DataFrame(db_scan_best_results)


In [ ]:
df_scan_best_results

Here it can be seen why the silouette score is difficult because it is often undefined.

In [ ]:
## save the combinations of parameters, which maximized the silouette score.

df_dbscan_localization.to_csv(config["dbscan_hyperparam_grid"])
df_scan_best_results

In [ ]:
df_scan_best_results.rename(columns={"dbscan_count": "dbscan_count_sil" }, inplace=True)

df_comparison = df_comparison.merge(df_scan_best_results, on='image_name', how='left')

In [ ]:

df_comparison = df_comparison.drop(["dbscan_noise", "dbscan_silouette_score", "eps", "min_samples", "dbscan_BIC_score", "with_noise", "bic_avg"], axis=1, errors="ignore")
df_comparison

### HDBSCAN clustering for each image


In [ ]:
from zooniverse.analysis import hdbscan

hdbscan_values = []

eps_variants = [0.0] # 0 is the default
min_cluster_sizes = [5] # 5 is the default


for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
    annotations_count = get_mark_overview(df_image_name)
    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"])
    
    # if less than min_cluster_sizes points are available clustering makes no sense
    if df_image_name.shape[0] >= 5: # If num_samples is 5 for the min_cluster_size is 5 there is no point in passing data with less than 5 samples
        params = [(eps, min_cluster_size, max_cluster_size) 
                    for eps in eps_variants
                    for min_cluster_size in min_cluster_sizes
                    for max_cluster_size in [None]
              ]

        df_hdbscan = hdbscan(df_marks=df_image_name[["x", "y"]],
                                     output_path=output_plot_path,
                                     plot=show_plots,
                                     show=show_plots,
                                     image_name=image_name,
                                     params=params)
        hdbscan_values.append(df_hdbscan)


df_hdbscan = pd.concat(hdbscan_values)



In [ ]:
df_hdbscan.drop(["with_noise"], axis=1, inplace=True)
df_hdbscan

In [ ]:
df_comparison = df_comparison.merge(df_hdbscan, on='image_name', how='left')
df_comparison.fillna(0, inplace=True)
df_comparison

In [ ]:
df_comparison.to_csv(config["comparison_dataset"])
print(f"saved {config['comparison_dataset']}")

# A look into the results

## The sum of the clustering
What is the sum of the methods

In [ ]:

df_comparison_sum = df_comparison[["median_count", "mean_count", "mode_count", "dbscan_count_sil", "HDBSCAN_count"]].sum().sort_values()
df_comparison_sum.to_csv(config["method_sums"])


In [ ]:
print(f"phase_tag: {phase_tag}, user_threshold: {user_threshold}")

## Compare the numbers
The counts are only for images which were in the dataset after filtering.

### Sum of all the Methods

In [ ]:
print(f"{config['method_sums'].name}")
df_method_sums = pd.read_csv(config["method_sums"])
df_method_sums

### Comparison per Image Level

In [ ]:
print(f"load {config['comparison_dataset']}")
pd.read_csv(config["comparison_dataset"])